Baixando dependências

In [12]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib

Importando dependências para o projeto

In [1]:
import cv2 as cv
import numpy as np

Função de player de vídeo:

In [2]:
captura = cv.VideoCapture('../videos/parkour.mp4')
if not captura.isOpened():
    print("Error ao abrir o arquivo de vídeo. Finalizando...")
    exit()

while True:
    ret, frame = captura.read()
    if not ret:
        print("Não pode receber frame. Finalizando...")
        break
    # Mostra o frame capturado
    cv.imshow('Frame', frame)
    # Verifica se a tecla 'q' foi pressionada
    if cv.waitKey(30) & 0xFF == ord('q'):
        break
    # Verifica se o botão de fechar (x) foi clicado
    if cv.getWindowProperty('Frame', cv.WND_PROP_VISIBLE) < 1:
        break
    
captura.release()
cv.destroyAllWindows()


Função de Detecção de Movimento

In [2]:
def detectar_movimento(video_path):
    captura = cv.VideoCapture(video_path)

    if not captura.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Lê o primeiro quadro do vídeo
    ret, frame_anterior = captura.read()
    if not ret:
        print("Não foi possível ler o vídeo.")
        return

    # Converte o primeiro quadro para escala de cinza
    frame_anterior = cv.cvtColor(frame_anterior, cv.COLOR_BGR2GRAY)
    # Aplica um filtro Gaussiano para suavizar o quadro
    frame_anterior = cv.GaussianBlur(frame_anterior, (21, 21), 0)

    while True:
        # Lê o próximo quadro
        ret, frame_atual = captura.read()
        if not ret:
            break

        # Converte o quadro atual para escala de cinza
        frame_atual_gray = cv.cvtColor(frame_atual, cv.COLOR_BGR2GRAY)
        # Aplica um filtro Gaussiano para suavizar o quadro
        frame_atual_gray = cv.GaussianBlur(frame_atual_gray, (21, 21), 0)

        # Calcula a diferença absoluta entre o quadro atual e o anterior
        diferenca = cv.absdiff(frame_anterior, frame_atual_gray)

        # Aplica uma limiarização (threshold) para destacar áreas de movimento
        _, limiar = cv.threshold(diferenca, 25, 255, cv.THRESH_BINARY)

        # Cria uma imagem vermelha do mesmo tamanho do frame original
        vermelho = np.zeros_like(frame_atual)
        vermelho[:, :] = [0, 0, 255]  # Define cor vermelha para toda a imagem (BGR)

        # Sobrepõe os pixels vermelhos nas áreas onde houve movimento
        movimento = cv.bitwise_and(vermelho, vermelho, mask=limiar)

        # Combina a imagem original com a sobreposição vermelha
        resultado = cv.addWeighted(frame_atual, 1, movimento, 1, 0)

        # Exibe o quadro atual com as detecções de movimento
        cv.imshow("Detecção de Movimento", resultado)

        # Atualiza o frame anterior para o próximo loop
        frame_anterior = frame_atual_gray
        
        # Sai do loop se a tecla 'q' for pressionada ou se a janela for fechada
        if cv.waitKey(30) & 0xFF == ord('q') or cv.getWindowProperty('Detecção de Movimento', cv.WND_PROP_VISIBLE) < 1:
            break

    captura.release()
    cv.destroyAllWindows()
detectar_movimento('../videos/parkour.mp4')